In [ ]:
%reset

In [ ]:
from os import listdir
from os.path import exists
from importlib import reload
import numpy as np
import pandas as pd
import pyxdf
import mne
from utils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
import time
import datetime
from datetime import datetime, timezone
import pickle
import plotly.express as px

print('Imports done...')

In [ ]:
data_path = 'C:/Users/tumfart/Code/github/master-thesis/data/'
#data_path = 'C:/Users/peter/Google Drive/measurements/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
# = 'A03'
paradigm = 'paradigm' # 'eye', 'paradigm'
plot = False
mne.set_log_level('WARNING')

trial_type_markers = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']

# Create path list for each subject:
paths = [str(data_path + subject + '/' + paradigm) for subject in subjects]

# 1. 2-class classification:

## 1.1. Cue-based distance

In [ ]:
mne.set_log_level('INFO')

# Set the epoch type
epoch_type = 'cue aligned 2 class long v short'

# Iterate over each subject and extract the streams
start = time.time()
for subject, path in zip(subjects, paths):
    print(f'Reading last fif file for subject {subject}', end=' ')
    file_names = [f for f in listdir(path) if '_bad_annotations_raw.fif' in f]

    # Load file
    raw = load_raw_file(dirpath=path, file=file_names[0])

    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Select subset of event_dict with following markers:

    markers_of_interest = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']

    event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

    epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=0.0, tmax=7.0, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg'], reject=dict(eeg=200e-6 ))

    # Downsample to 10 Hz:
    epochs = epochs.copy().resample(10)

    # Save epochs:
    store_name = path + '/' + subject + '_' + paradigm + '_epo.fif'
    epochs.save(store_name, overwrite=True)

    print()

mne.set_log_level('WARNING')
print(f'Finished epoching, took me {round(time.time() - start)} seconds...')

### 1.1.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    longs = [m for m in markers_of_interest if '-l' in m]
    shorts = [m for m in markers_of_interest if '-s' in m]
    epochs_long = epochs[longs]
    epochs_short = epochs[shorts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_long.get_data(), epochs_short.get_data()])
    y = np.concatenate([np.zeros(len(epochs_long)), np.ones(len(epochs_short))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': bool(False), 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 1.1.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    longs = [m for m in markers_of_interest if '-l' in m]
    shorts = [m for m in markers_of_interest if '-s' in m]
    epochs_long = epochs[longs]
    epochs_short = epochs[shorts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_long.get_data(), epochs_short.get_data()])
    y = np.concatenate([np.zeros(len(epochs_long)), np.ones(len(epochs_short))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)
        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 1.2. Movment-onset-based distance

In [ ]:
mne.set_log_level('INFO')

# Set the epoch type
epoch_type = 'movement onset aligned 2 class long v short'

# Iterate over each subject and extract the streams
start = time.time()
for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}', end=' ')
    file_names = [f for f in listdir(path) if '_bad_annotations_raw.fif' in f]

    # Load file
    raw = load_raw_file(dirpath=path, file=file_names[0])

    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Select subset of event_dict with following markers:

    # Looking at indication release (movement onset):
    trial_type = trial_type_markers
    period = ['i'] # 'i', 'c' .. indication, cue
    position = ['l', 'r', 't', 'b', 'c']
    state = ['1'] # 0,1 .. touch/release
    markers_of_interest = generate_markers_of_interest(trial_type, period, position, state)

    event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

    epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=-2.0, tmax=3.5, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg'], reject=dict(eeg=200e-6 ))

    # Downsample to 10 Hz:
    epochs = epochs.copy().resample(10)

    # Save epochs:
    store_name = path + '/' + subject + '_' + paradigm + '_epo.fif'
    epochs.save(store_name, overwrite=True)

    print()

mne.set_log_level('WARNING')
print(f'Finished epoching, took me {round(time.time() - start)} seconds...')

### 1.2.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifiying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    longs = [m for m in markers_of_interest if '-l' in m]
    shorts = [m for m in markers_of_interest if '-s' in m]
    epochs_long = epochs[longs]
    epochs_short = epochs[shorts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_long.get_data(), epochs_short.get_data()])
    y = np.concatenate([np.zeros(len(epochs_long)), np.ones(len(epochs_short))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 1.2.1. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    longs = [m for m in markers_of_interest if '-l' in m]
    shorts = [m for m in markers_of_interest if '-s' in m]
    epochs_long = epochs[longs]
    epochs_short = epochs[shorts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_long.get_data(), epochs_short.get_data()])
    y = np.concatenate([np.zeros(len(epochs_long)), np.ones(len(epochs_short))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

# 2. 4-class classification

## 2.1. Cue-based direction (all)

In [ ]:
mne.set_log_level('INFO')

# Set the epoch type
epoch_type = 'cue aligned 4 class direction (all)'

# Iterate over each subject and extract the streams
start = time.time()
for subject, path in zip(subjects, paths):
    print(f'Reading last fif file for subject {subject}', end=' ')
    file_names = [f for f in listdir(path) if '_bad_annotations_raw.fif' in f]

    # Load file
    raw = load_raw_file(dirpath=path, file=file_names[0])

    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Select subset of event_dict with following markers:

    markers_of_interest = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']

    event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

    epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=0.0, tmax=7.0, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg'], reject=dict(eeg=200e-6 ))

    # Downsample to 10 Hz:
    epochs = epochs.copy().resample(10)

    # Save epochs:
    store_name = path + '/' + subject + '_' + paradigm + '_epo.fif'
    epochs.save(store_name, overwrite=True)

    print()

mne.set_log_level('WARNING')
print(f'Finished epoching, took me {round(time.time() - start)} seconds...')

### 2.1.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'cue aligned 4 class direction (all)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BT' in m]
    downs = [m for m in markers_of_interest if 'TT' in m]
    lefts = [m for m in markers_of_interest if 'RT' in m]
    rights = [m for m in markers_of_interest if 'LT' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.1.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BT' in m]
    downs = [m for m in markers_of_interest if 'TT' in m]
    lefts = [m for m in markers_of_interest if 'RT' in m]
    rights = [m for m in markers_of_interest if 'LT' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 2.3. Cue-based direction (short)

### 2.2.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'cue aligned 4 class direction (short)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-s' in m]
    downs = [m for m in markers_of_interest if 'TTB-s' in m]
    lefts = [m for m in markers_of_interest if 'RTL-s' in m]
    rights = [m for m in markers_of_interest if 'LTR-s' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.2.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-s' in m]
    downs = [m for m in markers_of_interest if 'TTB-s' in m]
    lefts = [m for m in markers_of_interest if 'RTL-s' in m]
    rights = [m for m in markers_of_interest if 'LTR-s' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 2.3. Cue-based direction (long)

### 2.3.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'cue aligned 4 class direction (long)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-l' in m]
    downs = [m for m in markers_of_interest if 'TTB-l' in m]
    lefts = [m for m in markers_of_interest if 'RTL-l' in m]
    rights = [m for m in markers_of_interest if 'LTR-l' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.3.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-l' in m]
    downs = [m for m in markers_of_interest if 'TTB-l' in m]
    lefts = [m for m in markers_of_interest if 'RTL-l' in m]
    rights = [m for m in markers_of_interest if 'LTR-l' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 2.4. Movement-onset--based direction (all)

In [ ]:
mne.set_log_level('INFO')

# Set the epoch type
epoch_type = 'movement onset aligned 2 class long v short'

# Iterate over each subject and extract the streams
start = time.time()
for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}', end=' ')
    file_names = [f for f in listdir(path) if '_bad_annotations_raw.fif' in f]

    # Load file
    raw = load_raw_file(dirpath=path, file=file_names[0])

    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Select subset of event_dict with following markers:

    # Looking at indication release (movement onset):
    trial_type = trial_type_markers
    period = ['i'] # 'i', 'c' .. indication, cue
    position = ['l', 'r', 't', 'b', 'c']
    state = ['1'] # 0,1 .. touch/release
    markers_of_interest = generate_markers_of_interest(trial_type, period, position, state)

    event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

    epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=-2.0, tmax=3.5, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg'], reject=dict(eeg=200e-6 ))

    # Downsample to 10 Hz:
    epochs = epochs.copy().resample(10)

    # Save epochs:
    store_name = path + '/' + subject + '_' + paradigm + '_epo.fif'
    epochs.save(store_name, overwrite=True)

    print()

mne.set_log_level('WARNING')
print(f'Finished epoching, took me {round(time.time() - start)} seconds...')

### 2.4.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'movement onset aligned 4 class direction (all)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BT' in m]
    downs = [m for m in markers_of_interest if 'TT' in m]
    lefts = [m for m in markers_of_interest if 'RT' in m]
    rights = [m for m in markers_of_interest if 'LT' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.4.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BT' in m]
    downs = [m for m in markers_of_interest if 'TT' in m]
    lefts = [m for m in markers_of_interest if 'RT' in m]
    rights = [m for m in markers_of_interest if 'LT' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 2.5. Movement-onset--based direction (short)

### 2.5.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'movement onset aligned 4 class direction (short)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-s' in m]
    downs = [m for m in markers_of_interest if 'TTB-s' in m]
    lefts = [m for m in markers_of_interest if 'RTL-s' in m]
    rights = [m for m in markers_of_interest if 'LTR-s' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.5.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-s' in m]
    downs = [m for m in markers_of_interest if 'TTB-s' in m]
    lefts = [m for m in markers_of_interest if 'RTL-s' in m]
    rights = [m for m in markers_of_interest if 'LTR-s' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

## 2.6. Movement-onset--based direction (long)

### 2.6.1. Single timepoint

In [ ]:
start = time.time()

df_scores = create_scores_df()

# Set the epoch type
epoch_type = 'movement onset aligned 4 class direction (long)'

for subject, path in zip(subjects, paths):
    print(f'Classifying subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-l' in m]
    downs = [m for m in markers_of_interest if 'TTB-l' in m]
    lefts = [m for m in markers_of_interest if 'RTL-l' in m]
    rights = [m for m in markers_of_interest if 'LTR-l' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2]
    for tp in range(X.shape[2]):
        x = X[:,:,tp]

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': False, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)

        if tp != X.shape[2]-1:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(0, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [False]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

### 2.6.2. Windowed

In [ ]:
# 5 timestamps classifier:
start = time.time()

df_scores = create_scores_df()

for subject, path in zip(subjects, paths):
    print(f'Classifying for subject {subject}')
    file_names = [f for f in listdir(path) if 'epo.fif' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    epochs = mne.read_epochs(file, preload=True)

    # Get condition:
    ups = [m for m in markers_of_interest if 'BTT-l' in m]
    downs = [m for m in markers_of_interest if 'TTB-l' in m]
    lefts = [m for m in markers_of_interest if 'RTL-l' in m]
    rights = [m for m in markers_of_interest if 'LTR-l' in m]
    epochs_up = epochs[ups]
    epochs_down = epochs[downs]
    epochs_right = epochs[rights]
    epochs_left = epochs[lefts]

    # Create data matrix X (epochs x channels x timepoints) and label vector y (epochs x 1):
    X = np.concatenate([epochs_up.get_data(), epochs_down.get_data(), epochs_right.get_data(), epochs_left.get_data()])
    y = np.concatenate([np.zeros(len(epochs_up)), np.ones(len(epochs_down)), 2*np.ones(len(epochs_right)), 3*np.ones(len(epochs_left))])

    clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    n_len = X.shape[2] - 5
    for tp in range(5,X.shape[2]):
        x = X[:,:,tp-5:tp+1]
        x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))

        scores = cross_val_score(clf, x, y, cv=LeaveOneOut(), n_jobs=-1)

        # Add row to the dataframe:
        row_to_add = {'Timepoint': tp/10 + epochs.tmin, 'Accuracy': scores.mean(), 'Subject': subject, '5-point': True, 'Type': epoch_type, 'Init_marker': [markers_of_interest], 't_min': epochs.tmin, 't_max': epochs.tmax, 'epoch_info': [epochs.info], 'Date':datetime.now().strftime('%Y-%m-%d'), 'Time': datetime.now().strftime('%H:%M:%S')}
        df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)


        if tp != X.shape[2]+4:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}', end='\r')
        else:
            print(f'Measuring timestamp {tp+1}/{X.shape[2]}')

# Add mean of scores as subject: Mean:
# Add row to the dataframe:
row_to_add = {'Timepoint': (np.arange(5, X.shape[2])/10).tolist() + epochs.tmin, 'Accuracy': df_scores.groupby('Timepoint')['Accuracy'].mean().to_list(), 'Subject': ['Mean']*n_len, '5-point': [True]*n_len, 'Type': [epoch_type]*n_len, 'Init_marker': [markers_of_interest]*n_len, 't_min': [epochs.tmin]*n_len, 't_max': [epochs.tmax]*n_len, 'epoch_info': [[epochs.info]]*n_len, 'Date':[datetime.now().strftime('%Y-%m-%d')]*n_len, 'Time': [datetime.now().strftime('%H:%M:%S')]*n_len}
df_scores = pd.concat([df_scores, pd.DataFrame(row_to_add)], ignore_index=True)
# Store dataframe to full classification dataframe:
store_scores_df(df_scores)

print(f'Finished classification, took me {round(time.time() - start)} seconds...')

In [ ]:
df = pd.read_csv('classification_df.csv', index_col=0)

In [ ]:
fig = px.line(df, x='Timepoint', y='Accuracy', color='Subject', facet_col='5-point', facet_row='Type')
fig.show()
fig.write_html('classification_accuracies.html')